# **Retrieval Augmented Generation (RAG) 애플리케이션 구축: Part 1**

### 자신만의 문서를 활용하여 응답을 생성하는 애플리케이션 구축

- **Part 1**: RAG 개념을 소개하고, 최소한의 구현 방법을 설명합니다.  
- **Part 2** 기존 구현을 확장하여 대화형 상호작용과 다단계 검색 프로세스를 처리할 수 있도록 개선합니다.
---

## 전형적인 **RAG 애플리케이션**

두 가지 주요 구성 요소로 이루어져 있습니다:  

1. **인덱싱(Indexing)**: 데이터 소스를 수집하고 인덱싱하는 파이프라인입니다. *일반적으로 오프라인에서 수행됩니다.*  
2. **검색 및 생성(Retrieval and Generation)**: 실행 시간에 사용자 쿼리를 받아 인덱스에서 관련 데이터를 검색한 후, 모델에 전달하여 답변을 생성합니다.  

---

### **인덱싱 단계**  
일반적인 데이터 인덱싱 과정은 다음과 같습니다:  

1. **로드(Load)**  
   - 먼저 데이터를 불러와야 합니다. 이는 문서 로더(Document Loaders)를 사용하여 수행됩니다.  

2. **분할(Split)**  
   - 텍스트 분할기(Text Splitters)를 사용해 큰 `문서(Document)`를 작은 청크(chunk)로 나눕니다.  
   - 이렇게 하면 검색이 더 효율적이며, 모델의 제한된 컨텍스트 윈도우에 맞출 수 있습니다.  

3. **저장(Store)**  
   - 분할된 데이터를 저장하고 인덱싱할 장소가 필요합니다.  
   - 일반적으로 벡터 스토어(VectorStore)와 임베딩 모델(Embeddings)을 사용합니다.  
---

### **검색 및 생성 단계**  

일반적인 검색 및 생성 과정은 다음과 같습니다:  

4. **검색(Retrieve)**  
   - 사용자 입력을 받아 검색기(Retriever)를 사용하여 저장된 데이터에서 관련 청크를 검색합니다.  

5. **생성(Generate)**  
   - 챗 모델(ChatModel) 또는 LLM이 검색된 데이터를 포함한 프롬프트를 사용해 답변을 생성합니다.  


인덱싱이 완료된 후에는 LangGraph를 오케스트레이션 프레임워크로 사용하여 **검색 및 생성 단계**를 구현합니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
# 사용할 임베딩 모델의 이름을 지정

In [ ]:
# InMemoryVectorStore - 메모리 내에서 벡터 데이터를 저장 및 빠른 검색

이  노트북에서는 **웹사이트 콘텐츠에 대한 질문에 답변하는 애플리케이션**을 구축합니다.  
**텍스트를 로드, 분할, 인덱싱**한 후, **사용자 질문을 기반으로 관련 데이터를 검색**하고 답변을 생성합니다.

## **단계별 상세 설명**

## **1. 인덱싱 (Indexing)**

### **문서 불러오기 (Loading Documents)**

먼저 **Document Loaders** 중 **[WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base/)** 를 사용하여 블로그 게시물의 내용을 불러옵니다. 

* `WebBaseLoader`클라스는 내부적으로 `urllib`을 사용해 **웹 URL에서 HTML을 로드**합니다.
* 이후, `BeautifulSoup`을 사용해 **텍스트로 파싱**하고 **Document** 객체 목록으로 반환합니다.

#### **HTML → 텍스트 변환 커스터마이징**

* 우리는 `<h1>`, `<h2>`, `<h3>`, `<p>`, `<pre>`, `<li>` 등 주요 콘텐츠를 포함하는 태그만 추출하도록 설정합니다.
* 또한 일부 웹사이트에서는 User-Agent를 설정하지 않으면 콘텐츠를 차단할 수 있으므로, `requests_kwargs`를 사용해 User-Agent를 지정해줍니다.

In [ ]:
# 주요 콘텐츠 태그만 필터링 (제목, 본문, 코드 등)
# WebBaseLoader 사용: requests_kwargs로 User-Agent 설정
# 문서 로드
# 결과 확인

In [ ]:
# 문서 길이
# 첫번째 500 자 출력


---

### **문서 분할 (Splitting documents)**  

문서 길이가 언어 모델의 **컨텍스트 윈도우(context window)** 에 넣기에 너무 길 경우, 너무 긴 입력은 **정보를 효과적으로 찾아내기 어려울 수 있습니다.**  

이 문제를 해결하기 위해, **`Document`를 작은 청크(chunk)로 분할**하여 **임베딩(embedding)** 및 **벡터 저장(vector storage)** 에 사용합니다.  
이렇게 하면 블로그 게시물의 **가장 관련성 높은 부분만 검색**할 수 있습니다.  

---

**RecursiveCharacterTextSplitter**는 문서를 **공통 구분자(예: 줄바꿈)** 를  사용해 재귀적으로 분할합니다.  일반적인 텍스트 사용 사례에 가장 적합한 텍스트 분할기입니다.

In [ ]:
# 불러온 문서를 설정한 기준에 따라 청크로 분할
# 분할된 청크(서브 문서)의 개수 출력

### **문서 저장 (Storing documents)**

이제 분할된 **텍스트 청크**를 인덱싱해야 합니다. 이를 통해 검색할 수 있습니다.  

1. 각 **문서 청크**의 내용을 **임베딩(embedding)** 합니다.
2. 이 **임베딩을 벡터 스토어(Vector Store)** 에 삽입합니다.


In [ ]:
# 분할된 문서 청크(all_splits)는 임베딩되어 벡터 스토어에 저장됩니다.
# 반환값은 저장된 각 문서 청크의 고유 ID 목록입니다.
# 첫 세 개의 문서 ID를 출력합니다.

이로써 **인덱싱(Indexing)** 단계가 완료되었습니다!

- 이제 우리는 **질의 가능한 벡터 스토어**를 보유하고 있습니다.  
- 게시물의 청크가 저장되어 있으며, 사용자 질문을 받으면 **관련 청크를 반환**할 수 있습니다.  

---

## **2. 검색 및 생성 (Retrieval and Generation)**

이제 실제 **애플리케이션 로직(application logic)** 을 작성하여 다음과 같은 작업을 수행할 것입니다:  

1. **사용자 질문을 입력받기**  
2. **질문과 관련된 문서 청크 검색**  
3. **검색된 문서와 질문을 모델에 전달**  
4. **챗 모델(Chat Model)이 답변을 생성**  
---

### **프로세스 요약**  
1. **사용자 질문 입력 → 문서 검색 → 답변 생성**  
2. 관련 문서를 검색하고 질문과 함께 모델에 전달  
3. 모델이 **최종 답변**을 생성  

In [ ]:
# 사용 예시 1: format() - 문자열로 반환
# 사용 예시 2: invoke() - Runnable 로 반환

### **LangGraph**를 사용하여 **검색(Retrieval)** 과 **생성(Generation)** 단계를 하나의 애플리케이션으로 통합 

1. **입력 질문(question)**  
2. **검색된 문맥(retrieved context)**  
3. **생성된 답변(generated answer)**  

In [ ]:
# 애플리케이션의 상태(State) 객체 정의
class State(TypedDict):

#### **노드 (애플리케이션 단계)**

간단한 두 단계로 구성된 시퀀스를 정의해 보겠습니다:  

1. **검색 (Retrieval)**  
2. **생성 (Generation)**  

In [ ]:
# 사용자의 질문을 기반으로 벡터 스토어에서 관련 문서를 검색
def retrieve(state: State):
    # 벡터 스토어에서 질문과 유사도가 높은 문서를 검색
# 검색된 문서와 질문을 기반으로 모델이 답변을 생성
def generate(state: State):
    # 검색된 문서(context) 내용을 하나의 문자열로 합칩니다. -> 직렬화
    # 프롬프트에 질문과 직렬화된 문서 내용을 전달하여 메시지 생성

애플리케이션을 하나의 **`graph` 객체**로 컴파일합니다.  여기서는 **검색 단계**와 **생성 단계**를 **단일 시퀀스(sequence)** 로 연결합니다.

In [ ]:
# StateGraph 초기화
# 검색(retrieve)과 생성(generate) 단계를 순차적으로 실행하도록 설정 (node 자동 추가)
# 그래프의 시작점(START)을 'retrieve' 단계와 연결
# 그래프를 컴파일하여 최종 그래프 객체를 생성

In [ ]:
# 그래프를 호출하여 사용자 질문에 대한 답변을 생성
# 질문
# 문서의 출처
# 생성된 답변(Answer)을 출력

-----------------
**LangGraph**는 RAG 애플리케이션을 구축하는 데 반드시 필요하지는 않습니다. 실제로 개별 구성 요소를 사용하여 동일한 애플리케이션 로직을 구현할 수도 있습니다. 

In [ ]:
# Langgraph 없이 RAG 구현한 경우
# 벡터 스토어에서 질문과 유사한 문서 검색
# 검색된 문서들의 내용을 하나의 문자열로 결합
# 검색된 문서(Context)와 질문을 프롬프트 템플릿에 전달하여 메시지 생성
# LLM(대형 언어 모델)을 호출하여 답변 생성
# 생성된 답변 출력